# Python Bytecode and Disassembly
## Compiling Python Code and Code Objects
Usually the difference between compiled and interpreted languages is that a compiled language must be converted from high-level code to CPU instructions whereas interpreted languages read code line by line and execute it. However, Python is not fully interpreted like shells are nor is it compiled in the traditional sense we know. Instead Python source code is compiled to a simpler instruction set called bytecode which is executed by the Python Virtual Machine. This compilation step is usually performed at runtime as needed and is why you may `.pyc` files in a `__pycache__` folder after execution. This is where Python caches is compilation to use next execution if the code remains unchanged. It is possible to compile your Python files before execution using `python -m compileall [file]` and `.pyc` files can be run like normal Python files using `python [pyc_file]`.

To compile a string of Python code to a code object we can use the built in `compile` function. This gives us a code object which has different attributes like `co_code` which contains the bytecode in byte form, `co_consts` which contains all constants, and `co_varnames` which contains a list of variable names used in the code. Notice how any methods, classes, or lambda expressions within your code is compiled to seperate code objects stored within `co_consts`.

In [1]:
code = "print('hello world!')"
code = compile(code, 'test', 'exec')
code

<code object <module> at 0x7f90400503a0, file "test", line 1>

In [2]:
code.co_code

b'e\x00d\x00\x83\x01\x01\x00d\x01S\x00'

In [3]:
code.co_consts

('hello world!', None)

In [4]:
eval(code)

hello world!


## Disassembling Code
[`dis`](https://docs.python.org/3/library/dis.html) is a built-in module for disassembling CPython bytecode. The built in `dis.dis` method prints out the disassembly of any code object passed to it.

In [5]:
import dis
dis.dis(code)

  1           0 LOAD_NAME                0 (print)
              2 LOAD_CONST               0 ('hello world!')
              4 CALL_FUNCTION            1
              6 POP_TOP
              8 LOAD_CONST               1 (None)
             10 RETURN_VALUE


This disassembly format may be hard to read at first but from left to right is the line number in the code, byte offset, instruction name, and arguments. Python bytecode is run in a stack-based virtual machine and is oriented entirely around stack based data structures.
### Common Bytecode Instructions
Here are some of the common instructions you'll see in Python Bytecode
#### `POP_TOP`
Bytecode is run in a stack and this instruction removes the item at the top of the stack (usually refered to as top-of-stack or TOS)
#### `LOAD_NAME`
Pushes the value associated with the name onto the stack. Takes a single name argument.
#### `LOAD_CONST`
Pushes the constant argument onto the stack.
#### `LOAD_GLOBAL`
Pushes the value associated with the global name argument onto the stack. Works similarily to `LOAD_NAME`.
#### `STORE_FAST`
Stores the TOS item to the variable name given.
#### `CALL_FUNCTION`
Calls a callable/function item with positional arguments. The argument for this instruction is the number of positional arguments should be passed to the function from the TOS. Below the arguments is the callable item. This instruction will pop all the positional arguments and callable item off the stack and push the return value of the function.
#### `LOAD_METHOD`
Loads a method item with name matching argument from the TOS item. TOS is popped and the object and method are pushed to TOS if it is an unbound method. Otherwise, `NULL` and the method are pushed to TOS.
#### `CALL_METHOD`
Calls a method with positional arguments. The argument for this instruction is number of positional arguments passed from TOS. Below the arguments is the callable method and method object. Arguments, method, and method object are poped and the return value is pushed to TOS. This instruction is designed to be used with the `LOAD_METHOD` instruction.
#### `RETURN_VALUE`
Returns the TOS item to the caller of the function.